In [2]:
# Bring in all dependencies

import pandas as pd
import requests
import numpy as np
import json
import time
from datetime import date
from pprint import pprint
import sqlalchemy
#from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float
from config_poly import api_key
import sqlite3
#from splinter import Browser
#from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Establish link to Russell 100 index wiki page
russell_url = "https://en.wikipedia.org/wiki/Russell_1000_Index"

# Creaters browser window for scraping
#executable_path = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# Visits url, creates soup object
#browser.visit(russell_url)
#soup = BeautifulSoup(browser.html, 'html.parser')

In [5]:
thing = pd.read_html(russell_url)
company_table = thing[2]
companies = company_table["Company"]
myCompanies = pd.unique(companies)
companies

0                       2U
1                       3M
2      Abbott Laboratories
3              AbbVie Inc.
4                  Abiomed
              ...         
952                 Xilinx
953          XPO Logistics
954             Xylem Inc.
955     Yum China Holdings
956            Yum! Brands
Name: Company, Length: 957, dtype: object

In [6]:
tickers = company_table["Ticker"]
myTickers = tickers.drop_duplicates()
len(myTickers)

957

In [10]:
n = 100
newTickers = [tickers[i:i + n] for i in range(0, len(tickers), n)]
# Should be 10 lists
newTickers

[0     TWOU
 1      MMM
 2      ABT
 3     ABBV
 4     ABMD
       ... 
 95      AN
 96     AZO
 97     AVB
 98     AGR
 99     AVY
 Name: Ticker, Length: 100, dtype: object,
 100     AVT
 101     EQH
 102    AXTA
 103     AXS
 104     BKR
        ... 
 195    CTAS
 196    CSCO
 197     CIT
 198       C
 199     CFG
 Name: Ticker, Length: 100, dtype: object,
 200    CTXS
 201     CLH
 202     CLX
 203     CME
 204     CMS
        ... 
 295    ESRT
 296     EHC
 297     EGN
 298     ENR
 299     ETR
 Name: Ticker, Length: 100, dtype: object,
 300    EVHC
 301     EOG
 302    EPAM
 303     EPR
 304     EQT
        ... 
 395    HAIN
 396     HAL
 397     HBI
 398     THG
 399     HOG
 Name: Ticker, Length: 100, dtype: object,
 400     HIG
 401     HAS
 402      HE
 403     HCA
 404     HTA
        ... 
 495      LW
 496    LSTR
 497     LVS
 498     LAZ
 499     LEA
 Name: Ticker, Length: 100, dtype: object,
 500       LM
 501      LEG
 502     LDOS
 503      LEN
 504    LEN.B
        ...

In [7]:
# Initialize list to hold information from polygon.io
polygon_info = []
polygon_div = []

startTime = time.time()
#today = date.today()
#todays_date = today.strftime("%Y-%m-%d")

# Initializes counter to 0
count = 0

# Loops through list of lists of ticker symbols
for item in newTickers:
    
    # Loops through one list of ticker symbols
    for ticker in item:
        
        # Checks if count equals 4, used to prevent exceeding 
        # the API call limit of 5 calls/minute
        if count == 4:
            
            # Tells program to wait 1.5 minutes before running again
            time.sleep(90)

            # Ticker symbols to be used for API Call
            dataset_code = ticker

            # URLs for API Calls, the first for open and closed prices, 
            # the second for divident information
            url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
            div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

            # Saving responses from both urls
            response = requests.get(url).json()
            div_response = requests.get(div_url).json()

            # Adding responses to info and div lists, respectively
            polygon_info.append(response)
            polygon_div.append(div_response)

            # Resets counter
            count = 0

        else:
            # Ticker symbols to be used for API Call
            dataset_code = ticker

            # URLs for API Calls, the first for open and closed prices, 
            # the second for divident information
            url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
            div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

            # Saving responses from both urls
            response = requests.get(url).json()
            div_response = requests.get(div_url).json()

            # Adding responses to info and div lists, respectively
            polygon_info.append(response)
            polygon_div.append(div_response)

            # Increase counter by 2
            count += 2
    
    # Tells program to wait 1.5 minutes and a half before running again
    time.sleep(90)

executionTime = (time.time() - startTime)

ConnectionError: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v1/open-close/FNB/2021-04-23?unadjusted=true&apiKey=Spdxv4zfs4JKHfxIDy6yKANkGtphHgVp (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7faba0bbd490>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
realTime = executionTime/60
realTime

In [8]:
company_table = pd.DataFrame({'Name':myCompanies,"Symbol":myTickers})

In [9]:
polygon_info_df = pd.DataFrame(polygon_info)
del polygon_info_df['status']
del polygon_info_df['afterHours']
del polygon_info_df['preMarket']
polygon_info_df = polygon_info_df[['symbol','from','open','high','low','close','volume']]
polygon_info_df.columns = ['symbol','date','open','high','low','close','volume']

polygon_info_df

,symbol,date,open,high,low,close,volume
0,TWOU,2021-04-23,38.92,41.9500,38.510,40.56,1339545
1,MMM,2021-04-23,201.00,203.1550,200.990,202.20,1897775
2,ABT,2021-04-23,123.62,124.1100,122.645,123.31,5272865
3,ABBV,2021-04-23,109.71,111.7699,109.550,111.38,4671715
4,ABMD,2021-04-23,345.59,355.9400,344.790,351.03,291367
...,...,...,...,...,...,...,...
95,AN,2021-04-23,97.00,102.4050,96.600,102.01,913962
96,AZO,2021-04-23,1487.02,1487.0200,1451.770,1471.35,178930
97,AVB,2021-04-23,192.55,194.5300,192.075,193.88,568703
98,AGR,2021-04-23,52.07,52.0700,51.350,51.45,503512


In [10]:
all_div = polygon_div[1:]
div_amt_list = []
payDate = []
ticker_list = []

for div in all_div:
    result = div['results']
    
    if result == []:
        continue
        
    else:
        ticker = div["results"][0]['ticker']
        div_amt = div["results"][0]['amount']
        pay_date = div["results"][0]['paymentDate']
    
    ticker_list.append(ticker)
    div_amt_list.append(div_amt)
    payDate.append(pay_date)
    
polygon_div_df = pd.DataFrame({"Symbol":ticker_list,"Amount ($)":div_amt_list,"Payment Date":payDate})
polygon_div_df

,Symbol,Amount ($),Payment Date
0,MMM,1.480,2021-03-12
1,ABT,0.450,2021-02-16
2,ABBV,1.300,2021-02-16
3,ACN,0.880,2021-02-12
4,ATVI,0.410,2020-05-06
...,...,...,...
69,ADSK,0.015,2005-04-08
70,ADP,0.930,2021-04-01
71,AVB,1.590,2021-04-15
72,AGR,0.440,2021-04-01


In [11]:
polygon_info_df.to_csv('openClose.csv')
polygon_div_df.to_csv('dividend.csv')
company_table.to_csv('companies.csv')

In [30]:
polygon_info_df = pd.read_csv('openClose.csv')
polygon_div_df = pd.read_csv('dividend.csv')
company_table = pd.read_csv('companies.csv')

polygon_info_df.columns = ['index','symbol','date','opening','high','low','close','volume']
polygon_info_df = polygon_info_df.drop(columns='index')
polygon_div_df

,Unnamed: 0,Symbol,Amount ($),Payment Date
0,0,MMM,1.480,2021-03-12
1,1,ABT,0.450,2021-02-16
2,2,ABBV,1.300,2021-02-16
3,3,ACN,0.880,2021-02-12
4,4,ATVI,0.410,2020-05-06
...,...,...,...,...
69,69,ADSK,0.015,2005-04-08
70,70,ADP,0.930,2021-04-01
71,71,AVB,1.590,2021-04-15
72,72,AGR,0.440,2021-04-01


In [32]:
del polygon_div_df['Unnamed: 0']

KeyError: 'Unnamed: 0'

In [34]:
del company_table['Unnamed: 0']
company_table

,Name,Symbol
0,2U,TWOU
1,3M,MMM
2,Abbott Laboratories,ABT
3,AbbVie Inc.,ABBV
4,Abiomed,ABMD
...,...,...
95,AutoNation,AN
96,AutoZone,AZO
97,AvalonBay Communities,AVB
98,Avangrid,AGR


In [35]:
# Create sqlite database path
database_path = "russell_stock.sqlite"

# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")

In [36]:
# use engine to connect to existing tables/db
Base = declarative_base()
#conn = sqlite3.connect('russell_stock.sqlite')
#c = conn.cursor()

# Use this to clear out the db
# ----------------------------------
Base.metadata.drop_all(bind=engine)

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
#Base.metadata.create_all(engine)

In [37]:
# Define our General_Info table
class Open_Close(Base):
    __tablename__ = 'openClose'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    date = Column(String)
    opening = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    volume = Column(Integer)

# Define our Dividend table
class Dividend(Base):
    __tablename__ = 'dividend'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    amount = Column(Float)
    payment_date = Column(String)

# Define our Company Table
class Company(Base):
    __tablename__ = 'company'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    name = Column(String)

In [38]:
# Save references to each table (capital because they are considered classes) 
#Company = Database.classes.company
#Dividend = Database.classes.dividend
#Open_Close = Database.classes.open_close

In [39]:
#for i in polygon_info_df.index:
#    print(polygon_info_df['from'][i])


# insert dataframe into sqlite tables
company_table.to_sql(name='company', con=engine, if_exists='replace', index=True)
polygon_div_df.to_sql(name='dividend', con=engine, if_exists='replace', index=True)
polygon_info_df.to_sql(name='openClose', con=engine, if_exists='replace', index=True)

In [40]:
# # for i in polygon_info_df.index:
#     thisFrom = polygon_info_df["date"][i]
#     thisSymbol = polygon_info_df["symbol"][i]
#     thisOpen = polygon_info_df["open"][i]
#     thisHigh = polygon_info_df["high"][i]
#    thisLow = polygon_info_df["low"][i]
#    thisClose = polygon_info_df["close"][i]
#    thisVol = polygon_info_df["volume"][i]
    
    #row = [thisFrom,thisSymbol,thisOpen,thisHigh,thisLow,thisClose,thisVol]
    
#    thisCompany = Open_Close(date=thisFrom,symbol=thisSymbol,opening=thisOpen,high=thisHigh,low=thisLow,close=thisClose,volume=thisVol)
#    session = Session(bind=engine)
    #conn.execute('Insert into open_close values(?,?,?,?,?,?,?)', row)
    #engine.execute()
#    Session.configure(bind=engine)
#    session.add(thisCompany)
    #session.commit()
    
#session.close()
    
    
    
    

In [41]:
#c.close()
#session.commit()



In [42]:
#session.close()

#conn.close()

engine.dispose()

In [43]:
engine = create_engine(f"sqlite:///{database_path}")

connection = engine.connect()

# use engine to connect to existing tables/db
base = automap_base()
base.prepare(engine, reflect=True)

# View all of the classes/tables that automap found
thing = base.classes.keys()

print(thing)

# Save references to each table (capital because they are considered classes) 
#Company = Database.classes.company
#Dividend = Database.classes.dividend
inspector = inspect(engine)

[]


In [44]:
connection.close()

engine.dispose()